In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class Student(Base):
    __tablename__ = 'student'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

    def __str__(self):
        return f'This is {self.id} student {self.name}. Age: {self.age}'

    def __repr__(self):
        return f'This is {self.id} student {self.name}. Age: {self.age}'

class Subject(Base):
    __tablename__ = 'subject'

    id = Column(Integer, primary_key=True)
    name = Column(String)

    def __str__(self):
        return f'Subject {self.name}'

    def __repr__(self):
        return f'Subject {self.name}'

class StudentSubject(Base):
    __tablename__ = 'student_subject'

    id = Column(Integer, primary_key=True)
    student_id = Column(Integer, ForeignKey('student.id'))
    subject_id = Column(Integer, ForeignKey('subject.id'))

    student = relationship("Student")
    subject = relationship("Subject")

    def __str__(self):
        return f'Student {self.student_id} is enrolled in Subject {self.subject_id}'

    def __repr__(self):
        return f'Student {self.student_id} is enrolled in Subject {self.subject_id}'

DATABASE_URI = 'postgresql://{user}:{password}@{host}:{port}/{database}'


engine = create_engine(
    DATABASE_URI.format(
        host='dumbo.db.elephantsql.com',
        database='shipfodm',
        user='shipfodm',
        password='bJo0TT1z5RGalO3A7zWqLVnSC-mrsYFb',
        port=5432,
    )
)
Base.metadata.create_all(engine)


<ipython-input-1-78730f20a764>:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

student1 = Student(name='Alice', age=20)
student2 = Student(name='Bob', age=22)
student3 = Student(name='Carol', age=21)

subject1 = Subject(name='Math')
subject2 = Subject(name='English')
subject3 = Subject(name='History')

session.add_all([student1, student2, student3])
session.add_all([subject1, subject2, subject3])
session.commit()

enrollment1 = StudentSubject(student=student1, subject=subject2)
enrollment2 = StudentSubject(student=student2, subject=subject1)
enrollment3 = StudentSubject(student=student3, subject=subject2)
enrollment4 = StudentSubject(student=student3, subject=subject3)

session.add_all([enrollment1, enrollment2, enrollment3, enrollment4])
session.commit()

session.close()


In [4]:
Session = sessionmaker(bind=engine)
session = Session()

english_students = session.query(Student).join(StudentSubject).join(Subject).filter(Subject.name == 'English').all()

for student in english_students:
    print(student.name)

session.close()

Alice
Carol
